In [1]:
import pandas as pd
import numpy as np

In [2]:
import tqdm

In [3]:
baseData = pd.read_csv("../data/train_ver2.csv")

C:\Users\caose\AppData\Local\Temp\ipykernel_1448\3097669333.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  baseData = pd.read_csv("../data/train_ver2.csv")


In [4]:
### 3개월 데이터로 줄인다. 값이 너무 많아서 최근의 추세로 판단하겠다

In [ ]:
base1 = baseData.loc[baseData.fecha_dato == "2016-04-28"]

In [5]:
base2 = baseData.loc[baseData.fecha_dato == "2016-05-28"]

In [6]:
base3 = baseData.loc[baseData.fecha_dato == "2016-03-28"]

In [ ]:
data1 = base1.append(base2, ignore_index=True)

In [ ]:
data1 = data1.append(base3, ignore_index=True)

In [9]:
data1 = data1.reset_index(drop=True)

In [10]:
# 기존
data1 = data1.drop(['fecha_dato', 'ncodpers','ind_empleado','pais_residencia','fecha_alta','ind_nuevo','indrel','ult_fec_cli_1t',\
                    'conyuemp','canal_entrada','indfall','tipodom','cod_prov','nomprov','ind_actividad_cliente',\
                    'ind_aval_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',\
                    'ind_ctpp_fin_ult1','ind_ecue_fin_ult1','ind_hip_fin_ult1','indresi',\
                    'ind_reca_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1',\
                    'ind_cco_fin_ult1','ind_cder_fin_ult1',"antiguedad",'indrel_1mes','tiprel_1mes',\
                    'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_ahor_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1'], axis=1)

In [11]:
oneStepDf = data1.copy()

# 칼럼 처리

### 나이

In [12]:
oneStepDf["age"] = oneStepDf["age"].astype(int)

In [13]:
oneStepDf["age"] = np.where(oneStepDf.age <= 9,0,
                   np.where(oneStepDf.age <= 19,1,
                   np.where(oneStepDf.age <= 29,2,
                   np.where(oneStepDf.age <= 39,3,
                   np.where(oneStepDf.age <= 49,4,
                   np.where(oneStepDf.age <= 59,5,
                   np.where(oneStepDf.age <= 69,6,
                   np.where(oneStepDf.age <= 79,7,
                   np.where(oneStepDf.age <= 89,8,
                   np.where(oneStepDf.age <= 99,9,10))))))))))

### 총소득

In [14]:
oneStepDf["renta"].fillna(oneStepDf.groupby('age')["renta"].transform('median'), inplace=True)

In [15]:
oneStepDf["renta"] = oneStepDf["renta"].astype(int)

In [16]:
oneStepDf["renta"] = np.where(oneStepDf.renta <= 25000,1, # 2만5천
                     np.where(oneStepDf.renta <= 50000,2, # 5만
                     np.where(oneStepDf.renta <= 75000,3, # 7만5천
                     np.where(oneStepDf.renta <= 100000,4,5)))) #10만, 나머지 5로 분류

In [17]:
oneStepDf["renta"].drop_duplicates()

0     5
2     4
12    3
14    2
90    1
Name: renta, dtype: int32

### 성별

In [18]:
oneStepDf.sexo.drop_duplicates()

0           H
2           V
284454    NaN
Name: sexo, dtype: object

In [19]:
oneStepDf.sexo = oneStepDf.sexo.fillna("H")

In [20]:
oneStepDf.sexo.drop_duplicates()

0    H
2    V
Name: sexo, dtype: object

In [21]:
dict = {"V":0, "H":1}

oneStepDf.sexo = oneStepDf.sexo.str.upper()
oneStepDf.sexo = oneStepDf.sexo.map(dict)

In [22]:
oneStepDf.sexo = oneStepDf.sexo.astype(int)

In [23]:
oneStepDf.sexo.dtype

dtype('int32')

### 외국인 지수

In [24]:
dict = {"N":0, "S":1}

oneStepDf.indext = oneStepDf.indext.str.upper()
oneStepDf.indext = oneStepDf.indext.map(dict)

In [25]:
oneStepDf.indext.unique()

array([0, 1], dtype=int64)

### 고객 분류

In [26]:
oneStepDf.segmento.drop_duplicates()

0      02 - PARTICULARES
4               01 - TOP
43                   NaN
59    03 - UNIVERSITARIO
Name: segmento, dtype: object

In [27]:
oneStepDf.segmento = np.where(oneStepDf.segmento == '03 - UNIVERSITARIO', 0, 
                     np.where(oneStepDf.segmento == '01 - TOP', 1,
                     np.where(oneStepDf.segmento == '02 - PARTICULARES', 1, oneStepDf.segmento)))

In [28]:
oneStepDf.loc[oneStepDf["age"] < 2, 'segmento'] = oneStepDf.segmento.fillna(0)

In [29]:
oneStepDf.loc[oneStepDf["age"] >= 2, 'segmento'] = oneStepDf.segmento.fillna(1)

In [30]:
oneStepDf.segmento = oneStepDf.segmento.astype(int)

In [31]:
oneStepDf.segmento.dtype

dtype('int32')

In [32]:
oneStepDf.dtypes

sexo                 int32
age                  int32
indext               int64
renta                int32
segmento             int32
ind_cno_fin_ult1     int64
ind_ctju_fin_ult1    int64
ind_dela_fin_ult1    int64
ind_fond_fin_ult1    int64
ind_tjcr_fin_ult1    int64
ind_valo_fin_ult1    int64
dtype: object

In [33]:
oneStepDf = oneStepDf.reset_index(drop=True)

In [34]:
# label이 될 컬럼을 상품명으로 변경

oneStepDf.loc[oneStepDf['ind_dela_fin_ult1'] == 1, 'ind_dela_fin_ult1'] = '예금'
oneStepDf.loc[oneStepDf['ind_cno_fin_ult1'] == 1, 'ind_cno_fin_ult1'] = '급여계좌'
oneStepDf.loc[oneStepDf['ind_ctju_fin_ult1'] == 1, 'ind_ctju_fin_ult1'] = '주니어계좌'
oneStepDf.loc[oneStepDf['ind_fond_fin_ult1'] == 1, 'ind_fond_fin_ult1'] = '펀드계좌'
oneStepDf.loc[oneStepDf['ind_tjcr_fin_ult1'] == 1, 'ind_tjcr_fin_ult1'] = '신용카드'
oneStepDf.loc[oneStepDf['ind_valo_fin_ult1'] == 1, 'ind_valo_fin_ult1'] = '증권'
# oneStepDf.loc[oneStepDf['ind_deco_fin_ult1'] == 1, 'ind_deco_fin_ult1'] = '단기예금'
# oneStepDf.loc[oneStepDf['ind_deme_fin_ult1'] == 1, 'ind_deme_fin_ult1'] = '중기예금'
# oneStepDf.loc[data1['ind_ahor_fin_ult1'] == 1, 'ind_ahor_fin_ult1'] = '저축계좌'
# oneStepDf.loc[oneStepDf['ind_plan_fin_ult1'] == 1, 'ind_plan_fin_ult1'] = '연금'
# oneStepDf.loc[oneStepDf['ind_pres_fin_ult1'] == 1, 'ind_pres_fin_ult1'] = '대출'

In [35]:
oneStepDf

,sexo,age,indext,renta,segmento,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_dela_fin_ult1,ind_fond_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1
0,1,6,0,5,1,0,0,0,0,0,0
1,1,4,0,5,1,0,0,0,0,0,0
2,0,4,0,4,1,0,0,0,0,0,0
3,0,4,0,5,1,급여계좌,0,0,0,신용카드,0
4,1,4,0,5,1,0,0,예금,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1856524,0,2,0,4,0,0,0,0,0,0,0
1856525,1,2,0,5,0,0,0,0,0,0,0
1856526,0,2,0,5,0,0,0,0,0,0,0
1856527,0,2,0,5,0,0,0,0,0,0,0


In [36]:
oneStepDf.columns

Index(['sexo', 'age', 'indext', 'renta', 'segmento', 'ind_cno_fin_ult1',
       'ind_ctju_fin_ult1', 'ind_dela_fin_ult1', 'ind_fond_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1'],
      dtype='object')

In [37]:
# 라벨부분 컬럼
expandColumn = ['ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_dela_fin_ult1',\
                'ind_fond_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1']

In [38]:
# 컬럼
dataCol = ['sexo', 'age', 'indext', 'renta', 'segmento']

In [39]:
oneStepDf

,sexo,age,indext,renta,segmento,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_dela_fin_ult1,ind_fond_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1
0,1,6,0,5,1,0,0,0,0,0,0
1,1,4,0,5,1,0,0,0,0,0,0
2,0,4,0,4,1,0,0,0,0,0,0
3,0,4,0,5,1,급여계좌,0,0,0,신용카드,0
4,1,4,0,5,1,0,0,예금,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1856524,0,2,0,4,0,0,0,0,0,0,0
1856525,1,2,0,5,0,0,0,0,0,0,0
1856526,0,2,0,5,0,0,0,0,0,0,0
1856527,0,2,0,5,0,0,0,0,0,0,0


### 11개 상품 1개의 컬럼으로 수정

In [40]:
wholeList = []
for i in tqdm.tqdm(range(len(oneStepDf))):
    for j in expandColumn:
        rowList = []
        rowList.append(oneStepDf[j][i])
        for k in dataCol:
            rowList.append(oneStepDf[k][i])
        wholeList.append(rowList)
secondStepDf = pd.DataFrame(wholeList)

100%|██████████████████████████████████████████████████████████████████████| 1856529/1856529 [07:14<00:00, 4277.05it/s]


In [41]:
secondStepDf.rename(columns={0:"FINANCIALPRODUCT"}, inplace=True)
secondStepDf.rename(columns={1:"GENDER"}, inplace=True)
secondStepDf.rename(columns={2:"AGE"}, inplace=True)
secondStepDf.rename(columns={3:"CUST_TYPE"}, inplace=True)
secondStepDf.rename(columns={4:"INCOME"}, inplace=True)
secondStepDf.rename(columns={5:"FOREINERINDEX"}, inplace=True)

secondStepDf

,FINANCIALPRODUCT,GENDER,AGE,CUST_TYPE,INCOME,FOREINERINDEX
0,0,1,6,0,5,1
1,0,1,6,0,5,1
2,0,1,6,0,5,1
3,0,1,6,0,5,1
4,0,1,6,0,5,1
...,...,...,...,...,...,...
11139169,0,1,2,1,1,0
11139170,0,1,2,1,1,0
11139171,0,1,2,1,1,0
11139172,0,1,2,1,1,0


In [42]:
secondStepDf = secondStepDf.loc[secondStepDf.FINANCIALPRODUCT != 0]
secondStepDf = secondStepDf.reset_index(drop=True)
secondStepDf

,FINANCIALPRODUCT,GENDER,AGE,CUST_TYPE,INCOME,FOREINERINDEX
0,급여계좌,0,4,0,5,1
1,신용카드,0,4,0,5,1
2,예금,1,4,0,5,1
3,예금,0,3,0,5,1
4,신용카드,0,3,0,5,1
...,...,...,...,...,...,...
364417,급여계좌,0,5,0,5,1
364418,급여계좌,1,3,0,4,1
364419,주니어계좌,0,1,0,3,1
364420,급여계좌,1,2,0,5,0


In [ ]:
----------------------------------------------------------------------------------------------------------------

In [43]:
secondStepDf.to_csv("../test/Financial_Product_Recommendation.csv", index=False, mode ="w")